In [7]:
from numpy import *
from pylab import *
import string

from datetime import date
import imp
AgilentSCPI=imp.load_source('AgilentSCPI','c:/Users/Jacob/Nonlinearity/Codes/AgilentSCPI.py')
#from 'c:/Users/Jacob/Nonlinearity/Codes/' import AgilentSCPI
import time
import numpy as np
import sys
import matplotlib.pyplot as plt
import time

In [9]:
delayvec=np.arange(0,100,1)
n_aves=4096
t_delay=n_aves*2e-3 #2 ms between pulses


#addtodelay=45 #sample 2; or 1 and 2
#addtodelay=43.5 #sample 3; or 1 and or 2
addtodelay=48 #sampleA_COS; or 1 and or 2


print(t_delay)
fname_root="c:/Users/Jacob/Nonlinearity/Data/SampleB/2020-09-14/"
AMP=[10.0]
Ntrials=10

8.192


In [10]:
class timedata:

    def __init__(self, name):
        self.name = name
        self.time = np.array    # creates a new empty list
        self.data = np.array    # creates a new empty list


In [11]:
def readlabdat(fname,dataset):
# read in data from the scope and make a matrix of values from it
    
    
    f = open(fname,'r')
    #header1=f.readline()
    #header2=f.readline()
    tmp1=[]
    tmp2=[]
    
    for line in f:
        columns = line.split(',')
        tmp1.append(float(columns[0]))
        tmp2.append(float(columns[1]))

    dataset.time=np.asarray(tmp1)
    dataset.data=np.asarray(tmp2)
    f.close()
        

In [13]:
# read data from oscilloscope and save to file
def saveWave(filename,chs,npts='MAXIMUM'):
    for ch in chs:
        p1,x1 = osc.OscRead2Numpy(ch,npts)
        if 'data' in locals():
            data = vstack((data,x1))
        else:
            data = x1
            
    
    t = arange(len(x1))*p1[4] + p1[5]
    data = vstack((t,data)).T
    savetxt(filename, data, fmt="%12.6G", delimiter=',')
    print(shape(data), "data points saved to "+filename)

Setup Function Generator

In [ ]:
funcgen = AgilentSCPI.SCPI("134.153.36.84")
q=funcgen.query('*IDN?\n')
print q
setupFile="STATE_COSJ20"
cmd = ":MMEMory:LOAD:STATe 'INT:\\"+setupFile+"'\n"
funcgen.s.send(cmd)
# funcgen.s.send(":SOUR2:VOLT"+" "+str(AMP[0])+"\n")
# funcgen.s.send(":SOUR1:VOLT"+" 5.0"+"\n")
#this sets up the amplitudes and frequency of the pump (on channel 2) and probe (on channel 1)
funcgen.s.send(":SOUR2:VOLT"+" "+str(AMP[0])+"\n")
funcgen.s.send(":SOUR2:FREQ"+" "+str(50000)+"\n")

funcgen.s.send(":SOUR1:VOLT"+" "+str(1)+"\n")
funcgen.s.send(":SOUR1:FREQ"+" "+str(500000)+"\n")

Setup Oscilloscope

In [ ]:
osc = AgilentSCPI.SCPI("134.153.36.86")
q2=osc.query('*IDN?\n')
print q2
osc.reset()
osc.s.send(":TRIGger:EDGE:SOURCe EXTernal\n")
osc.s.send(":TRIGger:EDGE:LEVel 5.0\n")
osc.s.send(":TRIGger:EDGE:SLOPe POSitive\n")
osc.s.send(":TIM:REF LEFT\n")
osc.s.send(":TIM:DEL "+ str(addtodelay)+"us\n")
osc.s.send(":TIM:SCAL 1.0us\n")  #time scale 
osc.s.send(":CHANnel1:SCALe 0.01\n") # Vertical axis limit of probe 
osc.s.send(":CHANnel2:SCALe 20\n") #Vertical axis limit of pump
osc.s.send(":ACQuire:TYPE AVERage\n")
osc.s.send(":ACQuire:COUNt 4096\n")
#fname=fname_root+"junk.csv"
#saveWave(fname,[1])

#Do not put anything in this box below the current line
# addtodelay_=48
# delay_=1
# cmd_=":TRIG:DEL "+str(delay_)+"us\n"
# funcgen.s.send(cmd_)
# funcgen.s.send(":SOUR1:VOLT"+" "+str(5)+"\n") #*****
# funcgen.s.send(":OUTP1 ON\n")#*****
# funcgen.s.send(":OUTP2 ON\n") #*****
# osc.s.send(":TIM:DEL "+ str(addtodelay_+delay_)+"us\n")#*****

In [ ]:
for itrial in range(Ntrials):
    for ampval in AMP:
        funcgen.s.send(":SOUR2:VOLT"+" "+str(ampval)+"\n")
        funcgen.s.send(":OUTP1 ON\n")
        funcgen.s.send(":OUTP2 ON\n")
        cmd=":TRIG:DEL "+str(0)+"us\n"
        funcgen.s.send(cmd)
        fname_root_add=fname_root+"Trial"+str(itrial+1)+"/"+str(ampval)+"V/"
        fname=fname_root_add+str("STrans_B4.csv")
        osc.s.send(":TIM:REF LEFT\n")
        osc.s.send(":TIM:SCAL 20us\n")
        osc.s.send(":TIM:DEL 30us\n")
        osc.s.send("CHANnel2:SCALe 25\n")
        saveWave(fname,[2])
        osc.s.send(":TIM:SCAL 1.0us\n")
        scaldel=1
        for delay in delayvec:
            #set delay
#            cmd=":TRIG2:DEL "+str(delay)+"us\n"
            cmd=":TRIG:DEL "+str(delay)+"us\n"
            funcgen.s.send(cmd)
    
            osc.s.send(":TIM:DEL "+ str(addtodelay+delay)+"us\n")
 #           osc.s.send(":TIM:DEL "+ str(addtodelay)+"us\n")

            #record probe only
            funcgen.s.send(":OUTP1 ON\n") #probe on
            funcgen.s.send(":OUTP2 OFF\n") #PUMP off

            time.sleep(t_delay)

            fname=fname_root_add+"pr"+str(int(scaldel*delay))+".csv"
            saveWave(fname,[1])

            #record PUMP only
            osc.s.send(":TIM:DEL "+ str(addtodelay+delay)+"us\n")
  #          osc.s.send(":TIM:DEL "+ str(addtodelay)+"us\n")
            funcgen.s.send(":OUTP1 OFF\n") #probe on
            funcgen.s.send(":OUTP2 ON\n") #PUMP off
            time.sleep(t_delay)

            fname=fname_root_add+"pu"+str(int(scaldel*delay))+".csv"
            saveWave(fname,[1])

            #record PUMP and probe
            osc.s.send(":TIM:DEL "+ str(addtodelay+delay)+"us\n")
            #osc.s.send(":TIM:DEL "+ str(addtodelay)+"us\n")
            funcgen.s.send(":OUTP1 ON\n") #probe on
            funcgen.s.send(":OUTP2 ON\n") #PUMP off
            time.sleep(t_delay)

            fname=fname_root_add+"pp"+str(int(scaldel*delay))+".csv"
            saveWave(fname,[1])

funcgen.s.send(":OUTP1 OFF\n")
funcgen.s.send(":OUTP2 OFF\n")

In [ ]:
cmd=":TRIG2:DEL "+str(delay)+"us\n"
funcgen.s.send(cmd)